In [ ]:
# ===============================================================
# PROJECT OVERVIEW — AI-Powered Marketing Content Generator
# ===============================================================
# This project builds an AI system to automatically generate
# marketing content for products in a fictional retail store.
#
# Core idea:
# 1. Select a product from a fictional catalog.
# 2. Choose the type of content to generate (Product Description or Ad Copy).
# 3. Use a free, secure generative AI model (google/flan-t5-large)
#    to produce creative, witty, and meaningful content.
#
# Example:
# - Product: Running Shoes
# - Content Type: Ad Copy
# - Output: "Step into comfort and style! Our Running Shoes are perfect for your daily jogs, turning heads as you go."
#
# ===============================================================
# STEP-BY-STEP EXPLANATION
# ===============================================================
# Step 1 — Install Dependencies
# Step 2 — Load Product Data
# Step 3 — Model Setup
# Step 4 — Prompt Engineering (improved with few-shot examples)
# Step 5 — Gradio Interface
# ===============================================================

# 1. Install Dependencies
!pip install --quiet transformers torch pandas gradio

# 2. Imports & Load Data
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import gradio as gr

# Path to product CSV
folder_path = "/content/fictional_retail_docs"
csv_path = os.path.join(folder_path, "fictional_retail_products.csv")

if not os.path.exists(csv_path):
    raise FileNotFoundError(f"Could not find {csv_path}. Please upload your CSV file first.")

# Load CSV
df = pd.read_csv(csv_path)
print("Loaded product catalog:")
print(df.head())

# 3. Load AI Model (Free & Secure)
model_name = "google/flan-t5-large"  # Free and secure
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 4. Content Generation Function with Enhanced Prompting
def generate_content(product_name, content_type):
    """
    Generates creative and meaningful marketing content
    (Product Description or Ad Copy) for the given product.
    """

    # Few-shot examples to guide tone & style
    examples_description = (
        "Example 1 — Product: Wireless Earbuds\n"
        "Description: Experience freedom and clarity with our Wireless Earbuds. "
        "Perfect for calls, workouts, or your daily commute — crystal-clear sound, all day long.\n\n"
        "Example 2 — Product: Coffee Mug\n"
        "Description: Start your day right with our classic Coffee Mug — durable, stylish, "
        "and perfect for your morning brew at home or the office.\n\n"
    )

    examples_ad = (
        "Example 1 — Product: Wireless Earbuds\n"
        "Ad Copy: 'No wires. No limits. Just pure sound freedom — all day long!'\n\n"
        "Example 2 — Product: Coffee Mug\n"
        "Ad Copy: 'Sip. Smile. Repeat. The mug that makes mornings magical!'\n\n"
    )

    # Choose prompt based on content type
    if content_type == "Product Description":
        prompt = (
            examples_description +
            f"You are a professional product copywriter. Write a detailed, creative, and engaging "
            f"product description for a fictional retail product called '{product_name}'. "
            "Highlight what makes it unique, its key features, benefits, and ideal use cases. "
            "Make it sound friendly, believable, and under 100 words."
        )

    elif content_type == "Ad Copy":
        prompt = (
            examples_ad +
            f"You are a skilled marketing copywriter. Create a short, catchy, and witty advertisement "
            f"for a fictional retail product called '{product_name}'. Use an enthusiastic and persuasive tone, "
            "include a creative tagline or slogan, and make it suitable for social media or digital marketing. "
            "Keep it under 50 words."
        )

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)

    # Generate output
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        top_p=0.92,
        temperature=0.8,
        eos_token_id=tokenizer.eos_token_id
    )

    # Decode and return text
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text.strip()

# 5. Gradio Interface
product_list = df["product_name"].tolist()
content_types = ["Product Description", "Ad Copy"]

def gradio_interface(product_name, content_type):
    return generate_content(product_name, content_type)

demo = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Dropdown(product_list, label="Select Product"),
        gr.Dropdown(content_types, label="Select Content Type")
    ],
    outputs=gr.Textbox(
        label="Generated Content",
        lines=10,
        placeholder="Your content will appear here...",
        max_lines=30
    ),
    title="AI-Powered Marketing Content Generator",
    description=("💡 **Instructions:** Select a product and content type below to generate AI-powered marketing text.\n\n"
                 "**Content Types:**\n"
                 "- 📝 *Product Description*: Detailed, creative explanation of the product.\n"
                 "- 🎯 *Ad Copy*: Short, catchy marketing text (like a tagline or social media post).\n\n"
                 "⚠️ All products and content are fictional for demo purposes only.")
)

demo.launch(share=False)
